In [1]:
import json
import os
import sys
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%load_ext autoreload

%autoreload 2


In [4]:
import profiler
import single_model_profiles as sm_profs

In [24]:
profs = sm_profs.load_single_model_profiles()

In [26]:
profs.keys()

dict_keys(['alexnet', 'res152', 'res18', 'res50', 'inception', 'tf-kernel-svm', 'tf-lang-detect', 'tf-log-reg', 'tf-lstm', 'tf-nmt', 'tf-resnet-feats'])

In [ ]:
def load_results(results_dir):
    fs = os.listdir(results_dir)
    experiments = []
    for exp in fs:
        if exp[-4:] == "json":
            with open(os.path.join(results_dir, exp), "r") as f:
                data = json.load(f)
                experiments.append(data)
        else:
            # print("skipping %s" % os.path.join(results_dir, exp))
            pass
    return experiments

In [ ]:
var_size_results = load_results(
    os.path.abspath("../results/single_model_profs_old/kernel-svm-var-size/"))


mname, df = sm_profs.create_model_profile_df(None, var_size_results)




In [ ]:
df

In [ ]:
from sklearn.metrics.pairwise import rbf_kernel
x = np.random.rand(1,4096)
X = np.random.rand(10000,4096)
%timeit k = rbf_kernel(X,x)

In [ ]:
k.dot(x)